# 4. LangChain 기초


In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 4.1. LangChain 개요


### LangChain 설치


In [1]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 pydantic==2.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.1
    Uninstalling pydantic_core-2.33.1:
      Successfully uninstalled pydantic_core-2.33.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.2
    Uninstalling p

### LangSmith 설정


In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 4.2. LLM / Chat model


### LLM


In [9]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("안녕하세요. ")
print(ai_message)



안녕하세요. 반가워요. 저는 인공지능 언어모델 GPT-3입니다. 무엇을 도와드릴까요?


### Chat model


In [10]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요! 저는 존이라고 합니다!"),
    AIMessage(content="안녕하세요, 존님! 어떤 도움이 필요하신가요?"),
    HumanMessage(content="제 이름을 아시나요?"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

네, 존님이라고 말씀하셨습니다! 어떻게 도와드릴까요?


### 스트리밍


In [11]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요!"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

## 4.3. Prompt template


### PromptTemplate


In [12]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리의 레시피를 생각해 주세요.

요리명: {dish}""")

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 카레


#### ＜보충: 프롬프트 변수가 1개인 경우＞


In [13]:
prompt_value = prompt.invoke("카레")
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 카레


### ChatPromptTemplate


In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder


In [15]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요! 저는 존이라고 합니다!"),
            AIMessage("안녕하세요, 존님! 어떻게 도와드릴까요?"),
        ],
        "input": "제 이름을 아시나요?",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 저는 존이라고 합니다!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 존님! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='제 이름을 아시나요?', additional_kwargs={}, response_metadata={})]


### LangSmith의 Prompts


In [16]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("ychoi/recipe")

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


### (칼럼) 멀티모달 모델의 입력 처리


In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "이미지를 설명해 주세요."},
                {"type": "image_url", "image_url": {"url": "{image_url}"}},
            ],
        ),
    ]
)
image_url = "https://raw.githubusercontent.com/ychoi-kr/langchain-book/main/cover.jpg"

prompt_value = prompt.invoke({"image_url": image_url})

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

이 이미지는 책 표지입니다. 제목은 "챗GPT와 랭체인을 활용한 LLM 기반 AI 앱 개발"입니다. 표지에는 벌 모양의 종이접기 도안이 그려져 있으며, 노란색과 파란색 줄무늬가 특징입니다. 책은 랭체인과 LLM 애플리케이션 구축에 관한 내용을 다루고 있는 것으로 보입니다.


## 4.4. Output parser


### PydanticOutputParser를 사용한 Python 객체 변환


In [18]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [19]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [20]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [22]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "감자 2개",
    "당근 1개",
    "카레 가루 3큰술",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 4컵"
  ],
  "steps": [
    "닭고기를 한 입 크기로 자르고, 소금과 후추로 간을 한다.",
    "양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.",
    "팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶는다.",
    "닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.",
    "감자와 당근을 넣고 함께 볶는다.",
    "물 4컵을 붓고 끓인다.",
    "끓기 시작하면 불을 줄이고, 카레 가루를 넣고 잘 섞는다.",
    "약한 불에서 20분 정도 끓여서 재료가 부드러워질 때까지 조리한다.",
    "완성된 카레를 밥과 함께 서빙한다."
  ]
}


In [24]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['닭고기를 한 입 크기로 자르고, 소금과 후추로 간을 한다.', '양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.', '팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶는다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.', '감자와 당근을 넣고 함께 볶는다.', '물 4컵을 붓고 끓인다.', '끓기 시작하면 불을 줄이고, 카레 가루를 넣고 잘 섞는다.', '약한 불에서 20분 정도 끓여서 재료가 부드러워질 때까지 조리한다.', '완성된 카레를 밥과 함께 서빙한다.']


### StrOutputParser


In [25]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요. 저는 AI 어시스턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요. 저는 AI 어시스턴트입니다.


## 4.5. Chain—LangChain Expression Language(LCEL) 개요


### prompt와 model 연결


In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [27]:
chain = prompt | model

In [28]:
ai_message = chain.invoke({"dish": "카레"})
print(ai_message.content)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 이때 마늘과 생강을 추가하면 향이 더 좋아집니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### StrOutputParser를 연결에 추가


In [29]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "카레"})
print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 완두콩 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 마늘과 생강은 다져줍니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 투명해지면 다진 마늘과 생강을 넣고 볶다가 고기를 추가합니다. 고기가 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불로 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다. 원한다면 피망이나 완두콩을 추가하여 색감을 더할 수 있습니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성 후 먹으면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### PydanticOutputParser를 사용한 연결


In [30]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [32]:
chain = prompt_with_format_instructions | model | output_parser

In [33]:
recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['닭고기를 한 입 크기로 자르고, 소금과 후추로 간을 한다.', '양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.', '냄비에 식용유를 두르고 다진 양파를 넣어 볶는다.', '양파가 투명해지면 닭고기를 넣고 겉면이 익을 때까지 볶는다.', '감자와 당근을 넣고 함께 볶는다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓인다.', '끓기 시작하면 중약불로 줄이고 20분 정도 끓인다.', '재료가 부드러워지면 불을 끄고, 필요에 따라 간을 맞춘다.', '밥과 함께 따뜻하게 서빙한다.']


### （칼럼）with_structured_output


In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기', '양파', '감자', '당근', '카레 가루', '코코넛 밀크', '식용유', '소금', '후추', '물'] steps=['닭고기를 한 입 크기로 썰고, 소금과 후추로 간을 한다.', '양파를 다지고, 감자와 당근을 깍둑썰기로 준비한다.', '팬에 식용유를 두르고 다진 양파를 넣어 볶다가 투명해질 때까지 볶는다.', '닭고기를 넣고 겉면이 익을 때까지 볶는다.', '감자와 당근을 추가하고 함께 볶는다.', '카레 가루를 넣고 잘 섞은 후, 물과 코코넛 밀크를 붓고 끓인다.', '중불로 20-30분간 끓여서 재료들이 부드러워질 때까지 조리한다.', '소금으로 간을 맞추고, 원하는 농도로 졸여서 완성한다.']


## 4.6.LangChain의 RAG 관련 컴포넌트


### Document loader


In [35]:
!pip install langchain-community==0.3.0 GitPython==3.1.43

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.44
    Uninstalling GitPython-3.1.44:
      Successfully uninstalled GitPython-3.1.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0


In [36]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

412


### Document transformer


In [37]:
!pip install langchain-text-splitters==0.3.0

  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.22 requires langchain-text-splitters<1.0.0,>=0.3.7, but you have langchain-text-splitters 0.3.0 which is incompatible.


In [38]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

1421


### Embedding model


In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [40]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.006401920225471258, 0.008395073004066944, 0.023506203666329384, -0.02718920260667801, 0.035508450120687485, 0.006309845019131899, -0.0015436101239174604, 0.017299266532063484, 0.011471460573375225, -0.025694338604807854, -0.0010060546919703484, -0.002324893604964018, -0.009472891688346863, -0.008671297691762447, -0.015208622440695763, 0.04870225489139557, 0.007468906696885824, -0.015078634023666382, -0.029182355850934982, 0.0004945645923726261, 0.02539103291928768, 0.03301700949668884, -0.02643093839287758, 0.05056541785597801, 0.025824327021837234, -0.04740237072110176, -0.017440086230635643, 0.03477184846997261, -0.013128810562193394, -0.106936976313591, -0.01305298414081335, -0.05450839549303055, -0.025369368493556976, 0.03275703266263008, -0.015327777713537216, 0.052558571100234985, 0.04571252688765526, 0.00966245774179697, -0.011493125930428505, -0.019964024424552917, 0.012283886782824993, -0.004116293974220753, 0.015306113287806511, 0.00424086581915617, 0.0301572680473327

### Vector store


In [41]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:0

In [42]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

In [43]:
retriever = db.as_retriever()

In [44]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) service that automaticall

### LCEL을 사용한 RAG Chain 구현


In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 바탕으로 질문에 답변해 주세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

네, AWS S3에서 데이터를 읽어 들이기 위한 Document loader가 있습니다. `S3DirectoryLoader`와 `S3FileLoader`를 사용할 수 있습니다.
